In [1]:
import numpy as np
import pandas as pd
import requests
import json
import os
import ta

pd.set_option('display.max_columns', 100)


In [8]:
EXCHANGE_ID = "384d3f6f-c8ab-4552-969a-70fce9b1b242"

BTCUSDT_DAY_ID = "09fbb028-81c2-48dd-9077-d604f8ae856e"
BTCUSDT_HOUR_ID = ""
BTCUSDT_MINUTE_ID = ""

ETHUSDT_DAY_ID = "9d14a067-c116-4cb9-b33e-c8a5344a2337"
ETHUSDT_HOUR_ID = ""
ETHUSDT_MINUTE_ID = ""

BNBUSDT_DAY_ID = "d2ef1f0c-8842-4c75-8dc1-61e5410e511b"
BNBUSDT_HOUR_ID = ""
BNBUSDT_MINUTE_ID = ""




ASSET_ID = ETHUSDT_DAY_ID

base_url = "http://172.24.100.128:5000/"
# base_url = "http://localhost:5000/"


PROXIES = {
    
   'http': 'http://discproxy.virtual.uniandes.edu.co:443',
}


In [9]:
# Method to make API request to retrieve prices for a given asset ID between two specified Unix timestamps
def api_request_get_prices_between_unix_time(base_url, asset_id, unix_time_start, unix_time_end):

    # try:
    # Make GET request to API endpoint with query parameters for start and end Unix timestamps
    response = requests.get(base_url + "assets/" + asset_id + "/indicators_unix_between/",
                            params={'unix_time_start': unix_time_start, 'unix_time_end': unix_time_end},proxies=PROXIES)

    # Check if the request was successful
    if response.status_code == requests.codes.ok:
        # print("get prices between status code: "+str(response.status_code))
        # Convert JSON response to Python dictionary
        json_data = json.loads(response.content)
        # Normalize the dictionary and convert it to a Pandas DataFrame
        df = pd.json_normalize(json_data)

        # drop columns that are not needed
        df.drop(columns=['updated_at', 'created_at', 'date_time_gmt_5','ignore','asset_id'], inplace=True)

        # Return the DataFrame containing the retrieved price data
        return df

    else:
        # print("Failed api_request_get_prices_between_unix_time")
        print("get prices between status code: "+str(response.status_code))
        json_data = json.loads(response.content)
        print(json_data)
        return json_data
    


def api_request_get_asset_from_asset_id(base_url, exchange_id, asset_id):
    # try:
    # Make GET request to API endpoint

    response = requests.get(base_url + "exchanges/" +
                            exchange_id + "/asset/"+asset_id,proxies=PROXIES)

    # Check if the request was successful
    if response.status_code == requests.codes.ok:
        print("get asset status code: "+str(response.status_code))
        # Convert JSON response to Python dictionary
        json_data = json.loads(response.content)
        # Normalize the dictionary and convert it to a Pandas DataFrame
        df = pd.json_normalize(json_data)
        # Return the Unix timestamp of the last recorded price for the specified asset ID

        print("symbol: " + df["symbol"].iloc[0])
        print("interval: " + df["interval"].iloc[0])
        return df
    else:
        print(response.content)
        json_data = json.loads(response.content)
        df = pd.json_normalize(json_data)
        return df



# Method to make API request to retrieve prices for a given asset ID between two specified Unix timestamps
def api_request_get_all_indicators_from_price(base_url, price_id,):

    # try:
    # Make GET request to API endpoint with query parameters for start and end Unix timestamps
    response = requests.get(base_url + "prices/" + price_id + "/indicators/",proxies=PROXIES)

    # Check if the request was successful
    if response.status_code == requests.codes.ok:
        # Convert JSON response to Python dictionary
        json_data = json.loads(response.content)
        # Normalize the dictionary and convert it to a Pandas DataFrame
        df = pd.json_normalize(json_data)

        if df.empty:
            return df
        else:
        
            df.drop(columns=['id', 'created_at', 'updated_at','date_time_utc','unix_time','date_time_gmt_5'], inplace=True)

            df.set_index('price_id', inplace=True)

            df = df.replace(0, np.nan)

            return df

    else:
        print("get indicators status code: "+str(response.status_code))
        json_data = json.loads(response.content)
        print(json_data)
        return json_data
    

def get_indicators_and_merge(base_url,exchange_id, asset_id, unix_time_start, unix_time_end):


    # Get the asset from the API
    asset = api_request_get_asset_from_asset_id(base_url,exchange_id,asset_id)

    interval = asset["interval"].iloc[0]
    
    if interval == "minute":
        first_indicator_time = unix_time_start - 6000

    if interval == "hour":
        first_indicator_time = unix_time_start - 360000

    if interval == "day":
        first_indicator_time = unix_time_start - 8640000


    df = api_request_get_prices_between_unix_time(base_url, asset_id, first_indicator_time, unix_time_end)

    print(df.info())
    
    df_prices_wit_indicators_all = add_indicators(df)

    df_prices_wit_indicators = df_prices_wit_indicators_all.loc[df_prices_wit_indicators_all['unix_time'] >= unix_time_start]

    df_assets = pd.DataFrame()

    for i in df['id']:

        df2 = api_request_get_all_indicators_from_price(base_url, i)

        if not df2.empty:
            df_assets = pd.concat([df_assets, df2])
        
    if not df_assets.empty:
        df_merge = pd.merge(df_prices_wit_indicators, df_assets, left_on='id', right_on='price_id', how='left')
    else:
        df_merge = df_prices_wit_indicators

    return df_merge


# method to add the thecnic indicators to the dataframe
def add_indicators(df):

    # momentum
    df['ao'] = ta.momentum.awesome_oscillator(
        high=df['high_price'], low=df['low_price'])
    df['kama'] = ta.momentum.kama(close=df['close_price'])
    df['ppo'] = ta.momentum.ppo(close=df['close_price'])
    df['pvo'] = ta.momentum.pvo(volume=df['volume'])
    df['roc'] = ta.momentum.roc(close=df['close_price'])
    df['rsi'] = ta.momentum.rsi(close=df['close_price'])
    df['stochrsi'] = ta.momentum.stochrsi(close=df['close_price'])
    df['stoch'] = ta.momentum.stoch(
        close=df['close_price'], high=df['high_price'], low=df['low_price'])
    df['tsi'] = ta.momentum.tsi(close=df['close_price'])
    df['uo'] = ta.momentum.ultimate_oscillator(
        close=df['close_price'], high=df['high_price'], low=df['low_price'])
    df['wri'] = ta.momentum.williams_r(
        close=df['close_price'], high=df['high_price'], low=df['low_price'])

    # volume
    df['accdist'] = ta.volume.acc_dist_index(
        high=df['high_price'], low=df['low_price'], close=df['close_price'], volume=df['volume'])
    df['cmf'] = ta.volume.chaikin_money_flow(
        high=df['high_price'], low=df['low_price'], close=df['close_price'], volume=df['volume'])
    df['emv'] = ta.volume.ease_of_movement(
        high=df['high_price'], low=df['low_price'], volume=df['volume'])
    df['fi'] = ta.volume.force_index(
        close=df['close_price'], volume=df['volume'])
    df['mfi'] = ta.volume.money_flow_index(
        high=df['high_price'], low=df['low_price'], close=df['close_price'], volume=df['volume'])
    df['nvi'] = ta.volume.negative_volume_index(
        close=df['close_price'], volume=df['volume'])
    df['obv'] = ta.volume.on_balance_volume(
        close=df['close_price'], volume=df['volume'])
    df['smaemv'] = ta.volume.sma_ease_of_movement(
        high=df['high_price'], low=df['low_price'], volume=df['volume'])
    df['vpt'] = ta.volume.volume_price_trend(
        close=df['close_price'], volume=df['volume'])
    df['vwap'] = ta.volume.volume_weighted_average_price(
        high=df['high_price'], low=df['low_price'], close=df['close_price'], volume=df['volume'])

    # volatility
    df['atr'] = ta.volatility.average_true_range(
        high=df['high_price'], low=df['low_price'], close=df['close_price'])
    df['ulcer'] = ta.volatility.ulcer_index(close=df['close_price'])

    df['bbh'] = ta.volatility.bollinger_hband(close=df['close_price'])
    df['bbl'] = ta.volatility.bollinger_lband(close=df['close_price'])
    df['bbhi'] = ta.volatility.bollinger_hband_indicator(
        close=df['close_price'])
    df['bbli'] = ta.volatility.bollinger_lband_indicator(
        close=df['close_price'])
    df['bbmavg'] = ta.volatility.bollinger_mavg(close=df['close_price'])
    df['bb_pb'] = ta.volatility.bollinger_pband(close=df['close_price'])
    df['bb_wb'] = ta.volatility.bollinger_wband(close=df['close_price'])

    df['dchb'] = ta.volatility.donchian_channel_hband(
        close=df['close_price'], high=df['high_price'], low=df['low_price'])
    df['dclb'] = ta.volatility.donchian_channel_lband(
        close=df['close_price'], high=df['high_price'], low=df['low_price'])
    df['dcmb'] = ta.volatility.donchian_channel_mband(
        close=df['close_price'], high=df['high_price'], low=df['low_price'])
    df['dcpb'] = ta.volatility.donchian_channel_pband(
        close=df['close_price'], high=df['high_price'], low=df['low_price'])
    df['dcwb'] = ta.volatility.donchian_channel_wband(
        close=df['close_price'], high=df['high_price'], low=df['low_price'])

    df['kchb'] = ta.volatility.keltner_channel_hband(
        close=df['close_price'], high=df['high_price'], low=df['low_price'])
    df['kclb'] = ta.volatility.keltner_channel_lband(
        close=df['close_price'], high=df['high_price'], low=df['low_price'])
    df['kchbi'] = ta.volatility.keltner_channel_hband_indicator(
        close=df['close_price'], high=df['high_price'], low=df['low_price'])
    df['kclbi'] = ta.volatility.keltner_channel_lband_indicator(
        close=df['close_price'], high=df['high_price'], low=df['low_price'])
    df['kcmb'] = ta.volatility.keltner_channel_mband(
        close=df['close_price'], high=df['high_price'], low=df['low_price'])
    df['kcpb'] = ta.volatility.keltner_channel_pband(
        close=df['close_price'], high=df['high_price'], low=df['low_price'])
    df['kcwb'] = ta.volatility.keltner_channel_wband(
        close=df['close_price'], high=df['high_price'], low=df['low_price'])

    # trend

    # En dos de estos tres hay algo que esta botando warning
    df['adx'] = ta.trend.adx(close=df['close_price'],
                             high=df['high_price'], low=df['low_price'])
    df['adx_neg'] = ta.trend.adx_neg(
        close=df['close_price'], high=df['high_price'], low=df['low_price'])
    df['adx_pos'] = ta.trend.adx_pos(
        close=df['close_price'], high=df['high_price'], low=df['low_price'])

    df['aroon_up'] = ta.trend.aroon_up(close=df['close_price'])
    df['aroon_down'] = ta.trend.aroon_down(close=df['close_price'])

    df['cci'] = ta.trend.cci(close=df['close_price'],
                             high=df['high_price'], low=df['low_price'])
    df['dpo'] = ta.trend.dpo(close=df['close_price'])
    df['kst'] = ta.trend.kst(close=df['close_price'])
    df['kst_sig'] = ta.trend.kst_sig(close=df['close_price'])
    df['ema'] = ta.trend.ema_indicator(close=df['close_price'])

    df['ichimoku_a'] = ta.trend.ichimoku_a(
        high=df['high_price'], low=df['low_price'])
    df['ichimoku_b'] = ta.trend.ichimoku_b(
        high=df['high_price'], low=df['low_price'])
    df['ichimoku_base_line'] = ta.trend.ichimoku_base_line(
        high=df['high_price'], low=df['low_price'])
    df['ichimoku_conversion_line'] = ta.trend.ichimoku_conversion_line(
        high=df['high_price'], low=df['low_price'])

    df['macd'] = ta.trend.macd(close=df['close_price'])
    df['macd_diff'] = ta.trend.macd_diff(close=df['close_price'])
    df['macd_signal'] = ta.trend.macd_signal(close=df['close_price'])

    df['mi'] = ta.trend.mass_index(high=df['high_price'], low=df['low_price'])
    df['sma'] = ta.trend.sma_indicator(close=df['close_price'])
    df['wma'] = ta.trend.wma_indicator(close=df['close_price'])
    df['stc'] = ta.trend.stc(close=df['close_price'])
    df['trix'] = ta.trend.trix(close=df['close_price'])

    df['vi_pos'] = ta.trend.vortex_indicator_pos(
        close=df['close_price'], high=df['high_price'], low=df['low_price'])
    df['vi_neg'] = ta.trend.vortex_indicator_neg(
        close=df['close_price'], high=df['high_price'], low=df['low_price'])

    return df

In [10]:
df_prueba = get_indicators_and_merge(base_url, EXCHANGE_ID, ASSET_ID, 1681603200, 1681948800)
df_prueba.sort_values(by='unix_time', ascending=True, inplace=True)
df_prueba.tail(15)

get asset status code: 200
symbol: ETHUSDT
interval: day
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105 entries, 0 to 104
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   low_price        105 non-null    float64
 1   id               105 non-null    object 
 2   high_price       105 non-null    float64
 3   volume           105 non-null    float64
 4   unix_time        105 non-null    int64  
 5   qav              105 non-null    float64
 6   date_time_utc    105 non-null    object 
 7   num_trades       105 non-null    int64  
 8   taker_base_vol   105 non-null    float64
 9   open_price       105 non-null    float64
 10  taker_quote_vol  105 non-null    float64
 11  close_price      105 non-null    float64
dtypes: float64(8), int64(2), object(2)
memory usage: 10.0+ KB
None


/Users/julianoliverosforero/Desktop/2023-1/Tesis/cryptocurrency-market-prediction-project/data-loader/env/lib/python3.9/site-packages/ta/trend.py:780: RuntimeWarning: invalid value encountered in scalar divide
  dip[idx] = 100 * (self._dip[idx] / value)
/Users/julianoliverosforero/Desktop/2023-1/Tesis/cryptocurrency-market-prediction-project/data-loader/env/lib/python3.9/site-packages/ta/trend.py:785: RuntimeWarning: invalid value encountered in scalar divide
  din[idx] = 100 * (self._din[idx] / value)


,low_price,id,high_price,volume,unix_time,qav,date_time_utc,num_trades,taker_base_vol,open_price,taker_quote_vol,close_price,ao,kama,ppo,pvo,roc,rsi,stochrsi,stoch,tsi,uo,wri,accdist,cmf,emv,fi,mfi,nvi,obv,smaemv,vpt,vwap,atr,ulcer,bbh,bbl,bbhi,bbli,bbmavg,bb_pb,bb_wb,dchb,dclb,dcmb,dcpb,dcwb,kchb,kclb,kchbi,kclbi,kcmb,kcpb,kcwb,adx,adx_neg,adx_pos,aroon_up,aroon_down,cci,dpo,kst,kst_sig,ema,ichimoku_a,ichimoku_b,ichimoku_base_line,ichimoku_conversion_line,macd,macd_diff,macd_signal,mi,sma,wma,stc,trix,vi_pos,vi_neg,natural_gas_price,soybean_price,us_2_year_treasury_price,cotton_price,oats_price,usbond_price,gold_price,coffee_price,spy500_price,sugar_price,dow_jones_price,cocoa_price,nasdaq_price,rice_price,russell_2000_price,corn_price,us_10_year_treasury_price,silver_price,wheat_price,us_5_year_treasury_price
0,2072.72,942452c0-d9e8-41d3-a0a3-750982a81b9d,2141.54,319880.4393,1681603200,6.730346e+08,2023-04-16T00:00:00+00:00,529231,162518.1328,2090.61,3.419549e+08,2118.67,250.883618,1821.894171,5.509254,-5.407666,25.245031,66.194435,1.000000,96.298934,13.880141,62.969943,-3.701066,2.652208e+06,-0.017862,3.137967e+06,2.483152e+07,58.156759,1862.815092,3.638963e+06,776813.063010,60660.894388,1759.831874,132.435148,8.111654,2048.991951,1335.281049,1.0,0.0,1692.1365,1.097628,42.178093,2141.54,1285.44,1847.295,0.961138,32.630188,1761.870167,1621.838167,1.0,0.0,1691.854167,3.547988,8.276836,11.545666,24.436176,46.733199,100.0,20.0,206.293179,-84.3365,116.816293,31.936692,1823.139329,1773.8225,1699.655,1699.655,1847.990,95.196752,50.130689,45.066063,24.313474,1764.610000,1901.360000,99.890071,0.358163,1.023608,0.687162,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2056.25,8845b65d-0dbf-4f45-9aed-760f822ecca2,2120.51,426972.7071,1681689600,8.894333e+08,2023-04-17T00:00:00+00:00,674817,206218.4799,2118.66,4.296169e+08,2074.00,172.988676,1791.791021,3.308068,2.032427,9.743579,59.390411,0.725388,91.264308,6.955339,63.961423,-8.735692,3.902319e+06,0.114970,4.366423e+06,2.165539e+07,62.611945,1954.085434,3.491364e+06,461820.111536,47230.066102,1757.796675,173.394759,16.382696,2110.745483,1435.041517,0.0,0.0,1772.8935,0.945619,38.113060,2141.54,1368.39,1744.450,0.938161,43.168895,1846.876500,1691.467500,1.0,0.0,1769.172000,2.461457,8.784279,10.295669,28.319427,44.618453,60.0,72.0,162.551372,239.1165,93.502665,107.249428,1809.336361,1749.7075,1699.655,1754.965,1744.450,57.937473,20.014252,37.923221,25.375395,1796.117500,1824.986444,31.356320,0.339560,0.996545,0.721876,2.275,1517.00,102.972656,83.300003,342.00,129.75000,1994.199951,201.000000,4176.75,24.440001,34129.0,3003.0,13186.50,1733.0,1814.199951,676.50,114.359375,25.052999,889.75,109.101562
2,2051.00,fe3344f2-b81d-43cd-9cee-daad7b7aa63d,2125.00,414244.6135,1681776000,8.672915e+08,2023-04-18T00:00:00+00:00,636801,205943.0188,2073.99,4.311824e+08,2103.50,209.160676,1827.703084,4.818404,-1.151982,23.778981,57.801341,0.829944,94.665484,9.458514,64.107582,-5.334516,5.206435e+06,0.268352,-6.234481e+04,2.500707e+07,61.819560,1956.903154,3.800617e+06,545162.794547,1117.068095,1869.706286,165.510230,12.017738,2223.726617,1483.501383,0.0,0.0,1853.6140,0.837581,39.934163,2141.54,1368.39,1945.765,0.930982,18.877744,1925.860667,1764.516667,1.0,0.0,1845.188667,2.100997,8.744038,10.760156,30.148074,42.409782,28.0,40.0,108.195535,-79.7340,151.614839,113.195508,1935.721774,1850.3650,1699.655,1754.965,1945.765,88.983320,25.791696,63.191624,25.564253,1922.495833,1986.031556,99.618099,0.522796,0.923089,0.736963,2.366,1519.25,102.949219,84.550003,347.50,130.03125,2007.400024,205.250000,4180.00,24.540001,34124.0,3054.0,13193.00,1734.0,1806.000000,677.50,114.484375,25.247999,881.25,109.164062
3,1923.03,7c879db1-026b-40e0-8330-5f72a3fab0cf,2104.60,775389.0602,1681862400,1.547934e+09,2023-04-19T00:00:00+00:00,1116849,381226.9641,2103.51,7.604785e+08,1933.73,224.021912,1829.234163,4.437398,2.034602,9.011320,52.480563,0.260534,58.812722,8.803412,54.218229,-41.187278,4.522434e

In [ ]:
df_prueba.tail(15)